In [69]:
from mdr_RTMM import *

In [70]:
init()

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.conda.m1.cxx11thread.serialization.python312.Release 2024.39+release.59628fbc5bc09f1221e1642f1f8d157ce49b1410 2024-09-23T07:49:48] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.m1.cxx11thread.serialization.pyth

In [71]:
def rotamer_trials_mm(pose, seqpos, mutation, score_function, seqpos_resfile, minimize):
    """
    Identical to pack_rotamers() function but uses a different Mover.
    """
    
    # Clone the WT and MUT poses.
    wt_pose = pose.clone()
    mut_pose = pose.clone()
    
    print(f'Processing {seqpos_resfile} for position {seqpos}', flush=True)
    print(f'Native structure score: {score_pose(wt_pose, score_function)}', flush=True)
    
    # initialize TaskFactory
    tf = TaskFactory()
    tf.push_back(operation.InitializeFromCommandline())
    tf.push_back(operation.RestrictToRepacking())
    tf.push_back(operation.ExtraRotamersGeneric())
    tf.push_back(operation.NoRepackDisulfides())
    tf.push_back(operation.UseMultiCoolAnnealer(states=10))
    
    # read resfile
    parse_rf = operation.ReadResfile(seqpos_resfile)
    tf.push_back(parse_rf)
    
    # create a packer task
    wt_packer_task = tf.create_task_and_apply_taskoperations(wt_pose)
    
    # create and apply a mover
    wt_mover = pack_min.RotamerTrialsMinMover(score_function, wt_packer_task)
    wt_mover.apply(wt_pose)
    
    print(wt_mover.info(), flush=True)
    print(f'WT score after packing: {score_pose(wt_pose, score_function)}', flush=True)
    
    # Optionally minimize
    if minimize:
        minimize_sidechains(wt_pose, score_function)
        print(f'WT score after minimization: {score_pose(wt_pose, score_function)}', flush=True)
        
    # score WT
    score_wt = score_pose(wt_pose, score_function)
    
    # Repeat process for the mutant
    mutate_residue(mut_pose, seqpos, mutation)
    
    # create packer task and apply it to the MUT pose.
    mut_packer_task = tf.create_task_and_apply_taskoperations(mut_pose)
    mut_mover = pack_min.RotamerTrialsMinMover(score_function, mut_packer_task)
    mut_mover.apply(mut_pose)

    print(mut_mover.info(), flush=True)
    print(f'MUT score after packing: {score_pose(mut_pose, score_function)}', flush=True)
    
    # Optionally minimize
    if minimize:
        minimize_sidechains(mut_pose, score_function)
        print(f'MUT score after minimization: {score_pose(mut_pose, score_function)}', flush=True)
        
    # score the MUT
    score_mut = score_pose(mut_pose, score_function)
    
    # Calculate the dE
    delta_energy = score_mut - score_wt
    
    return delta_energy

In [72]:
# setup MDR as usual

environ['PDB_DIR'] = './RTMM_input'
environ['RF_DIR'] = './RTMM_resfiles'

In [73]:
pdb_house = environ.get('PDB_DIR')
resfile_house = environ.get('RF_DIR')

In [74]:
chain_to_mutate = 'A'

In [75]:
ref_pdb = path.join(pdb_house, '1QYS.pdb')

In [76]:
ref_string = parse_pdb(ref_pdb)

In [77]:
ref_pose = string_to_pose(ref_string)

core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading Selenium SE from MSE as SD from MET
core.io.pose_from_sfr.PoseFromSFRBuilder: Reading MSE as MET!
core.io.pose_from_sfr.PoseFromSFRBuilder: Adding undetected upper terminus type to residue 92,   94 A
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 13
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue LYS 13
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CE  on residue LYS 13
core.conformation.Conformation: [ WARNING ] missing heavya

In [78]:
# proceed doing just pack_rotamers function (equivalent)

ssm_mutations = create_ssm_dict(ref_pose, chain_to_mutate)

In [79]:
print(ref_pose)

PDB file name: 
Total residues: 92
Sequence: DIQVQVNIDDNGKNFDYTYTVTTESELQKVLNELMDYIKKQGAKRVRISITARTKKEAEKFAAILIKVFAELGYNDINVTFDGDTVTVEGQL
Fold tree:
FOLD_TREE  EDGE 1 92 -1 


In [80]:
# carry out packing function using PackRotamersMover and compare to RotamerTrialsMinMover

seqpos = 91
mutation = 'D'
minimize = False

de_prm = pack_rotamers(ref_pose, seqpos, mutation, score_function=pyrosetta.rosetta.core.scoring.get_score_function(is_fullatom=True), seqpos_resfile=path.join(resfile_house, f'{seqpos}.resfile'), minimize=minimize)


de_rtmm = rotamer_trials_mm(ref_pose, seqpos, mutation, score_function=pyrosetta.rosetta.core.scoring.get_score_function(is_fullatom=True), seqpos_resfile=path.join(resfile_house, f'{seqpos}.resfile'), minimize=minimize)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
Processing ./RTMM_resfiles/91.resfile for position 91
Native structure score: 206.2147741464288
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 41 rotamers at 3 positions.
core.pack.pack_rotamers: Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
std_list[REMARK PackingRes, 76 A, 90 A, 91 A, REMARK DesignRes]
WT score after packing: 198.05363767627762
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 7 rotamers at 1 positions.
core.pack.pack_rotamers: Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction

In [81]:
print(f'∆E with PackRotamerMover: {de_prm}')
print(f'∆E with PackRotamerMinMover: {de_rtmm}')

∆E with PackRotamerMover: -0.3413080644037336
∆E with PackRotamerMinMover: -0.3907161953818843
